# Modeling and the Temperature of the Earth with the Atmosphere using Absorption Spectra Data

In [34]:
# Import Libraries
import pandas as pd
import math
import numpy as np
import functools
import bisect
from tqdm import tqdm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from Unit0 import model_utils

In [35]:
# Constants
plancks_constant = 6.62607015e-34
speed_of_light = 299792458
boltzmann_constant = 1.380649e-23

In [36]:
# Define Calculation Functions
def plancks_law(wavelength, temperature):
    return 2 * plancks_constant * speed_of_light ** 2 / wavelength ** 5 / math.expm1(plancks_constant * speed_of_light / wavelength / boltzmann_constant / temperature)


def spectral_radiance(wavelength, temperature, transmittances: []):
    return plancks_law(wavelength, temperature) * functools.reduce(lambda x, y: x * y, transmittances)


def radiance(wavelengths, temperature, transmittances: []):
    delta_wavelength = wavelengths[1] - wavelengths[0]
    return functools.reduce(lambda x, y: x + y, [
        spectral_radiance(wavelength, temperature, transmittances) * delta_wavelength for wavelength in wavelengths])

In [37]:
# Load Data
data1 = pd.read_csv("Data/Absorption 0.1-3 Microns.csv")
data2 = pd.read_csv("Data/Absorption 3-12 Microns.csv")
data3 = pd.read_csv("Data/Absorption 12-40 Microns.csv")

In [38]:
# Process data into lists of tuples of wavelength and transmittance

# 0.1 to 3 microns
transmittance_1_h2o = list(zip(data1["H2O Wavelength (microns)"], data1["H2O Transmittance"]))
transmittance_1_co2 = list(zip(data1["CO2 Wavelength (microns)"], data1["CO2 Transmittance"]))
transmittance_1_ch4 = list(zip(data1["CH4 Wavelength (microns)"], data1["CH4 Transmittance"]))
transmittance_1_o3 = list(zip(data1["O3 Wavelength (microns)"], data1["O3 Transmittance"]))
transmittance_1_n2o = list(zip(data1["N2O Wavelength (microns)"], data1["N2O Transmittance"]))
transmittance_1_n2 = list(zip(data1["N2 Wavelength (microns)"], data1["N2 Transmittance"]))
transmittance_1_o2 = list(zip(data1["O2 Wavelength (microns)"], data1["O2 Transmittance"]))

# 3 to 12 microns
transmittance_2_h2o = list(zip(data2["Wavelength (microns)"], data2["H2O Transmittance"]))
transmittance_2_co2 = list(zip(data2["Wavelength (microns)"], data2["CO2 Transmittance"]))
transmittance_2_ch4 = list(zip(data2["Wavelength (microns)"], data2["CH4 Transmittance"]))
transmittance_2_o3 = list(zip(data2["Wavelength (microns)"], data2["O3 Transmittance"]))
transmittance_2_n2o = list(zip(data2["Wavelength (microns)"], data2["N2O Transmittance"]))
transmittance_2_n2 = list(zip(data2["Wavelength (microns)"], data2["N2 Transmittance"]))
transmittance_2_o2 = list(zip(data2["Wavelength (microns)"], data2["O2 Transmittance"]))

# 12 to 40 microns
transmittance_3_h2o = list(zip(data3["Wavelength (microns)"], data3["H2O Transmittance"]))
transmittance_3_co2 = list(zip(data3["Wavelength (microns)"], data3["CO2 Transmittance"]))
transmittance_3_ch4 = list(zip(data3["Wavelength (microns)"], data3["CH4 Transmittance"]))
transmittance_3_o3 = list(zip(data3["Wavelength (microns)"], data3["O3 Transmittance"]))
transmittance_3_n2o = list(zip(data3["Wavelength (microns)"], data3["N2O Transmittance"]))
transmittance_3_n2 = list(zip(data3["Wavelength (microns)"], data3["N2 Transmittance"]))
transmittance_3_o2 = list(zip(data3["Wavelength (microns)"], data3["O2 Transmittance"]))

# Combine the groups of wavelengths into one list for each component of absorption
transmittance_h2o = transmittance_1_h2o + transmittance_2_h2o + transmittance_3_h2o
transmittance_co2 = transmittance_1_co2 + transmittance_2_co2 + transmittance_3_co2
transmittance_ch4 = transmittance_1_ch4 + transmittance_2_ch4 + transmittance_3_ch4
transmittance_o3 = transmittance_1_o3 + transmittance_2_o3 + transmittance_3_o3
transmittance_n2o = transmittance_1_n2o + transmittance_2_n2o + transmittance_3_n2o
transmittance_n2 = transmittance_1_n2 + transmittance_2_n2 + transmittance_3_n2
transmittance_o2 = transmittance_1_o2 + transmittance_2_o2 + transmittance_3_o2

In [39]:
# Finds value in a less than or equal to x
def find_le(a, x, key):
    i = bisect.bisect(a, x, key=key)
    return a[i - 1 if i else 0]


# Finds value in a greater than or equal to x
def find_ge(a, x, key):
    i = bisect.bisect_left(a, x, key=key)
    return a[i if i < len(a) else i - 1]


# Interpolates value of x based on lower and upper bounds in a
def interpolate(a, x):
    key = lambda p: p[0]
    value = lambda p: p[1]
    p0 = find_le(a, x, key)
    p1 = find_ge(a, x, key)
    x0 = key(p0)
    x1 = key(p1)
    y0 = value(p0)
    if x0 == x1:
        return y0
    y1 = value(p1)
    return y0 + (y1 - y0) / (x1 - x0) * (x - x0)